# Python Básico: Importación de Datos

Este notebook cubre los conceptos básicos de Python necesarios para trabajar con datos, incluyendo:
- Conceptos básicos de Python
- Importación de datos desde archivos CSV
- Conexión a bases de datos MySQL
- Manipulación básica de datos con pandas

## 1. Importación de Librerías Necesarias

In [5]:
# Librerías esenciales para análisis de datos
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración para visualización
%matplotlib inline
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Librerías importadas correctamente")

Librerías importadas correctamente


## 2. Conceptos Básicos de Python

### 2.1 Variables y Tipos de Datos

In [6]:
# Variables básicas
nombre = "Mantenimiento Predictivo"
numero_activos = 150
probabilidad_falla = 0.75
es_critico = True

print(f"Tipo de 'nombre': {type(nombre)}")
print(f"Tipo de 'numero_activos': {type(numero_activos)}")
print(f"Tipo de 'probabilidad_falla': {type(probabilidad_falla)}")
print(f"Tipo de 'es_critico': {type(es_critico)}")

Tipo de 'nombre': <class 'str'>
Tipo de 'numero_activos': <class 'int'>
Tipo de 'probabilidad_falla': <class 'float'>
Tipo de 'es_critico': <class 'bool'>


### 2.2 Listas y Diccionarios

In [7]:
# Listas
activos = ["Motor A", "Motor B", "Compresor C"]
probabilidades = [0.2, 0.5, 0.8]

# Diccionarios
configuracion_db = {
    'host': '127.0.0.1',
    'database': 'palantir_maintenance',
    'user': 'root',
    'password': 'admin',
    'port': 3306
}

print(f"Lista de activos: {activos}")
print(f"Configuración DB: {configuracion_db}")

Lista de activos: ['Motor A', 'Motor B', 'Compresor C']
Configuración DB: {'host': '127.0.0.1', 'database': 'palantir_maintenance', 'user': 'root', 'password': 'admin', 'port': 3306}


## 3. Importación de Datos desde CSV

### 3.1 Leer un archivo CSV

In [8]:
# Ejemplo: Leer un archivo CSV
# Nota: Asegúrate de tener un archivo CSV en la ruta especificada
# df_csv = pd.read_csv('ruta/al/archivo.csv')

# Ejemplo con datos simulados
datos_ejemplo = {
    'asset_id': [1, 2, 3, 4, 5],
    'asset_name': ['Motor A', 'Motor B', 'Compresor C', 'Bomba D', 'Válvula E'],
    'failure_count': [2, 5, 1, 3, 0],
    'probability': [0.2, 0.8, 0.1, 0.5, 0.05]
}

df_ejemplo = pd.DataFrame(datos_ejemplo)
print("DataFrame creado desde diccionario:")
print(df_ejemplo)
print(f"\nForma del DataFrame: {df_ejemplo.shape}")
print(f"\nInformación del DataFrame:")
df_ejemplo.info()

DataFrame creado desde diccionario:
   asset_id   asset_name  failure_count  probability
0         1      Motor A              2         0.20
1         2      Motor B              5         0.80
2         3  Compresor C              1         0.10
3         4      Bomba D              3         0.50
4         5    Válvula E              0         0.05

Forma del DataFrame: (5, 4)

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   asset_id       5 non-null      int64  
 1   asset_name     5 non-null      object 
 2   failure_count  5 non-null      int64  
 3   probability    5 non-null      float64
dtypes: float64(1), int64(2), object(1)
memory usage: 288.0+ bytes


### 3.2 Operaciones Básicas con DataFrames

In [9]:
# Ver las primeras filas
print("Primeras 3 filas:")
print(df_ejemplo.head(3))

# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print(df_ejemplo.describe())

# Seleccionar columnas específicas
print("\nSolo columnas numéricas:")
print(df_ejemplo[['asset_id', 'failure_count', 'probability']])

# Filtrar datos
print("\nActivos con probabilidad > 0.3:")
print(df_ejemplo[df_ejemplo['probability'] > 0.3])

Primeras 3 filas:
   asset_id   asset_name  failure_count  probability
0         1      Motor A              2          0.2
1         2      Motor B              5          0.8
2         3  Compresor C              1          0.1

Estadísticas descriptivas:
       asset_id  failure_count  probability
count  5.000000       5.000000     5.000000
mean   3.000000       2.200000     0.330000
std    1.581139       1.923538     0.315436
min    1.000000       0.000000     0.050000
25%    2.000000       1.000000     0.100000
50%    3.000000       2.000000     0.200000
75%    4.000000       3.000000     0.500000
max    5.000000       5.000000     0.800000

Solo columnas numéricas:
   asset_id  failure_count  probability
0         1              2         0.20
1         2              5         0.80
2         3              1         0.10
3         4              3         0.50
4         5              0         0.05

Activos con probabilidad > 0.3:
   asset_id asset_name  failure_count  probabil

## 4. Conexión a Base de Datos MySQL

### 4.1 Configuración de Conexión

In [10]:
# Configuración de la base de datos MySQL
DB_CONFIG = {
    'host': '127.0.0.1',
    'database': 'palantir_maintenance',
    'user': 'root',
    'password': 'admin',
    'port': 3306
}

def conectar_mysql(config):
    """Función para conectar a MySQL"""
    try:
        connection = mysql.connector.connect(**config)
        if connection.is_connected():
            db_info = connection.get_server_info()
            print(f"Conectado a MySQL Server versión {db_info}")
            return connection
    except Error as e:
        print(f"Error al conectar a MySQL: {e}")
        return None

# Conectar a la base de datos
connection = conectar_mysql(DB_CONFIG)

Conectado a MySQL Server versión 9.4.0


### 4.2 Consultar Datos desde MySQL usando pandas

In [11]:
# Método 1: Usando mysql.connector y pandas
if connection and connection.is_connected():
    query = "SELECT * FROM assets LIMIT 10"
    df_assets = pd.read_sql(query, connection)
    print("Datos de la tabla 'assets':")
    print(df_assets.head())
    print(f"\nForma: {df_assets.shape}")
else:
    print("No hay conexión disponible. Verifica la configuración de la base de datos.")

Datos de la tabla 'assets':
   asset_id             asset_name  asset_type  \
0         1  Compressor Unit A-101  Compressor   
1         2     Pump Station B-205        Pump   
2         3    Conveyor Belt C-301    Conveyor   
3         4      HVAC System D-401        HVAC   
4         5        Generator E-501   Generator   

                         location installation_date manufacturer model_number  \
0            Building 1 - Floor 2        2020-01-15  Atlas Copco        GA-75   
1           Building 2 - Basement        2019-06-20     Grundfos        CR-32   
2  Building 3 - Production Line 1        2021-03-10      Siemens      SIMATIC   
3               Building 4 - Roof        2018-11-05      Carrier     30XA-100   
4         Building 5 - Power Room        2020-09-12      Cummins        QSK60   

        status          created_at          updated_at  
0  operational 2026-01-23 22:30:53 2026-01-23 22:30:53  
1  operational 2026-01-23 22:30:53 2026-01-23 22:30:53  
2  operationa

C:\Users\carlo\AppData\Local\Temp\ipykernel_10540\2332925807.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_assets = pd.read_sql(query, connection)


### 4.3 Consultar la Tabla faliure_probability_base

In [12]:
# Consultar la tabla faliure_probability_base
if connection and connection.is_connected():
    query = """
    SELECT * 
    FROM faliure_probability_base 
    LIMIT 10
    """
    df_failure_base = pd.read_sql(query, connection)
    print("Datos de la tabla 'faliure_probability_base':")
    print(df_failure_base.head())
    print(f"\nColumnas disponibles: {list(df_failure_base.columns)}")
    print(f"\nForma: {df_failure_base.shape}")
    print(f"\nInformación del DataFrame:")
    df_failure_base.info()
else:
    print("No hay conexión disponible.")

Datos de la tabla 'faliure_probability_base':
   base_id  asset_id     extraction_date  asset_age_days asset_status  \
0        1         1 2022-11-20 10:00:00            1040  operational   
1        2         2 2022-11-20 10:00:00            1219  operational   
2        3         3 2022-11-20 10:00:00             620  operational   
3        4         4 2022-11-20 10:00:00            1476  maintenance   
4        5         5 2022-11-20 10:00:00             800  operational   

   sensor_total_readings_30d  sensor_warning_count_30d  \
0                          4                         1   
1                          3                         0   
2                          2                         0   
3                          3                         0   
4                          3                         0   

   sensor_critical_count_30d  sensor_avg_normal_value  \
0                          0                     75.5   
1                          0                    125.

C:\Users\carlo\AppData\Local\Temp\ipykernel_10540\3891458873.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_failure_base = pd.read_sql(query, connection)


### 4.4 Método Alternativo: Usando SQLAlchemy (Recomendado)

In [13]:
# Método alternativo usando SQLAlchemy (más robusto)
from sqlalchemy import create_engine

# Crear string de conexión
connection_string = f"mysql+pymysql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

try:
    engine = create_engine(connection_string)
    
    # Consultar datos
    query = "SELECT * FROM faliure_probability_base LIMIT 10"
    df_sqlalchemy = pd.read_sql(query, engine)
    
    print("Datos obtenidos con SQLAlchemy:")
    print(df_sqlalchemy.head())
    
    # Cerrar conexión
    engine.dispose()
except Exception as e:
    print(f"Error con SQLAlchemy: {e}")
    print("Asegúrate de tener instalado: pip install sqlalchemy pymysql")

Datos obtenidos con SQLAlchemy:
   base_id  asset_id     extraction_date  asset_age_days asset_status  \
0        1         1 2022-11-20 10:00:00            1040  operational   
1        2         2 2022-11-20 10:00:00            1219  operational   
2        3         3 2022-11-20 10:00:00             620  operational   
3        4         4 2022-11-20 10:00:00            1476  maintenance   
4        5         5 2022-11-20 10:00:00             800  operational   

   sensor_total_readings_30d  sensor_warning_count_30d  \
0                          4                         1   
1                          3                         0   
2                          2                         0   
3                          3                         0   
4                          3                         0   

   sensor_critical_count_30d  sensor_avg_normal_value  \
0                          0                     75.5   
1                          0                    125.5   
2        

## 5. Limpieza y Preparación de Datos

### 5.1 Manejo de Valores Faltantes

In [14]:
# Si tenemos datos de la base de datos
if connection and connection.is_connected():
    query = "SELECT * FROM faliure_probability_base"
    df = pd.read_sql(query, connection)
    
    # Verificar valores faltantes
    print("Valores faltantes por columna:")
    print(df.isnull().sum())
    
    # Porcentaje de valores faltantes
    print("\nPorcentaje de valores faltantes:")
    print((df.isnull().sum() / len(df)) * 100)
    
    # Opciones para manejar valores faltantes:
    # 1. Eliminar filas con valores faltantes
    # df_clean = df.dropna()
    
    # 2. Llenar con valores específicos
    # df_filled = df.fillna(0)  # Llenar con 0
    
    # 3. Llenar con la media (solo columnas numéricas)
    # df_filled_mean = df.fillna(df.mean())
    
    print(f"\nForma original: {df.shape}")
    # print(f"Forma después de dropna: {df_clean.shape}")

Valores faltantes por columna:
base_id                      0
asset_id                     0
extraction_date              0
asset_age_days               0
asset_status                 0
sensor_total_readings_30d    0
sensor_warning_count_30d     0
sensor_critical_count_30d    0
sensor_avg_normal_value      0
sensor_avg_warning_value     6
sensor_avg_critical_value    8
sensor_max_value             0
sensor_min_value             0
sensor_std_value             0
failure_count_365d           0
failure_critical_count       0
failure_high_count           0
failure_medium_count         0
failure_low_count            0
failure_avg_downtime         0
failure_total_downtime       0
failure_unresolved_count     0
days_since_last_failure      0
task_total_365d              0
task_completed_count         0
task_in_progress_count       0
task_pending_count           0
task_avg_estimated_hours     0
task_avg_actual_hours        3
task_total_hours             0
days_since_last_task         3
order_to

C:\Users\carlo\AppData\Local\Temp\ipykernel_10540\3774400323.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


### 5.2 Cerrar Conexión

In [15]:
# Siempre cerrar la conexión cuando termines
if connection and connection.is_connected():
    connection.close()
    print("Conexión a MySQL cerrada correctamente")

Conexión a MySQL cerrada correctamente


## Resumen

En este notebook hemos aprendido:
1. ✅ Conceptos básicos de Python (variables, listas, diccionarios)
2. ✅ Importación de datos desde CSV usando pandas
3. ✅ Conexión a bases de datos MySQL
4. ✅ Consulta de datos desde MySQL
5. ✅ Operaciones básicas con DataFrames
6. ✅ Manejo de valores faltantes

**Próximos pasos:** En el siguiente notebook aprenderemos sobre modelos de clasificación.